# 04 - Complete Calibration Workflows

This notebook demonstrates complete calibration workflows for quantum devices in LeeQ.

## Learning Objectives
- Understand automated calibration procedures
- Learn calibration optimization strategies
- Practice with measurement calibration
- Explore calibration data management

## Prerequisites
- Complete [03_multi_qubit.ipynb](03_multi_qubit.ipynb)
- Understand quantum device characterization

## Setup and Configuration

In [ ]:
import leeq
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML, display
import json
import os
from datetime import datetime

# Import LeeQ calibration modules
from leeq.experiments.builtin.basic.calibrations import *
from leeq.experiments.builtin.basic.characterizations import SimpleT1, SpinEchoMultiLevel
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.chronicle import Chronicle, log_and_record
from leeq.experiments.experiments import ExperimentManager
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

print("✓ Calibration environment initialized")

# Initialize simulation environment
Chronicle().start_log()
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon
virtual_transmon = VirtualTransmon(
    name="VQubit",
    qubit_frequency=5040.4,
    anharmonicity=-198,
    t1=70,
    t2=35,
    readout_frequency=9645.4,
    quiescent_state_distribution=np.asarray([0.8, 0.15, 0.04, 0.01])
)

setup = HighLevelSimulationSetup(
    name='CalibrationSetup',
    virtual_qubits={2: virtual_transmon}
)
manager.register_setup(setup)

# Configuration for calibration
configuration = {
    'hrid': 'CalibQ',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.5,  # Initial guess
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

qubit = TransmonElement(name='CalibQ', parameters=configuration)
print("✓ Calibration qubit initialized")

## Automated Calibration Procedures

Automated calibration is essential for maintaining quantum device performance. This section demonstrates:

- **Sequential calibration workflow**: Systematic parameter optimization
- **Error handling**: Robust calibration with fallback procedures 
- **Parameter validation**: Ensuring calibrated values are physically reasonable
- **Optimization strategies**: Smart parameter adjustment algorithms

### Calibration Workflow Overview

1. **Device Setup & Validation**: Initialize devices and verify configuration
2. **Basic Parameter Calibration**: Rabi amplitude, frequency, phase, DRAG
3. **Coherence Characterization**: T1, T2 measurements for validation
4. **Parameter Optimization**: Fine-tune parameters for maximum fidelity
5. **Data Persistence**: Save calibration results to Chronicle logs
6. **Validation & Recovery**: Verify results and handle failures gracefully

In [ ]:
# Automated Calibration Implementation
print("=== Starting Automated Calibration Workflow ===\n")

class AutomatedCalibrationWorkflow:
    """Complete automated calibration workflow with error handling."""
    
    def __init__(self, qubit):
        self.qubit = qubit
        self.calibration_log = {
            'timestamp': datetime.now().isoformat(),
            'qubit_id': qubit.hrid,
            'initial_params': {},
            'calibrated_params': {},
            'validation_results': {}
        }
        
    def run_full_calibration(self):
        """Execute complete calibration sequence."""
        print("1. Initial Parameter Validation")
        self.validate_initial_params()
        
        print("\n2. Amplitude Calibration (Rabi)")
        self.calibrate_amplitude()
        
        print("\n3. Frequency Calibration (Ramsey)")
        self.calibrate_frequency()
        
        print("\n4. Phase Calibration (Pingpong)")
        self.calibrate_phase()
        
        print("\n5. DRAG Calibration")
        self.calibrate_drag()
        
        print("\n6. Coherence Validation")
        self.validate_coherence()
        
        print("\n7. Save Calibration Results")
        self.save_calibration()
        
        return self.calibration_log
    
    def validate_initial_params(self):
        """Validate initial device parameters."""
        params = self.qubit.get_c1('f01').get_parameters()
        self.calibration_log['initial_params'] = params
        
        print(f"  Initial frequency: {params['freq']:.2f} MHz")
        print(f"  Initial amplitude: {params['amp']:.4f}")
        print(f"  Initial DRAG: {params['alpha']:.2f}")
        
        # Basic sanity checks
        assert 4000 < params['freq'] < 6000, "Frequency out of reasonable range"
        assert 0 < params['amp'] < 1, "Amplitude out of range"
        print("  ✓ Initial parameters validated")
        
    def calibrate_amplitude(self):
        """Calibrate qubit drive amplitude using Rabi oscillation."""
        try:
            # Run Rabi experiment
            rabi = NormalisedRabi(
                dut_qubit=self.qubit,
                amp=self.qubit.get_c1('f01').get_parameters()['amp'],
                start=0.01,
                stop=0.3,
                step=0.01,
                update=True  # Auto-update qubit parameters
            )
            
            # Store calibrated amplitude
            new_amp = self.qubit.get_c1('f01').get_parameters()['amp']
            self.calibration_log['calibrated_params']['amplitude'] = new_amp
            print(f"  ✓ Calibrated amplitude: {new_amp:.4f}")
            
        except Exception as e:
            print(f"  ⚠ Amplitude calibration failed: {e}")
            print("  Using default amplitude")
            
    def calibrate_frequency(self):
        """Calibrate qubit frequency using Ramsey fringes."""
        try:
            # Multi-scale Ramsey for accurate frequency
            # Coarse scan
            ramsey_coarse = SimpleRamseyMultilevel(
                dut=self.qubit,
                set_offset=10,
                stop=0.5,
                step=0.01
            )
            
            # Fine scan
            ramsey_fine = SimpleRamseyMultilevel(
                dut=self.qubit,
                set_offset=1,
                stop=5,
                step=0.1
            )
            
            new_freq = self.qubit.get_c1('f01').get_parameters()['freq']
            self.calibration_log['calibrated_params']['frequency'] = new_freq
            print(f"  ✓ Calibrated frequency: {new_freq:.2f} MHz")
            
        except Exception as e:
            print(f"  ⚠ Frequency calibration failed: {e}")
            
    def calibrate_phase(self):
        """Calibrate phase using pingpong sequence."""
        try:
            pingpong = AmpPingpongCalibrationSingleQubitMultilevel(
                dut=self.qubit
            )
            
            self.calibration_log['calibrated_params']['phase_calibrated'] = True
            print("  ✓ Phase calibration complete")
            
        except Exception as e:
            print(f"  ⚠ Phase calibration failed: {e}")
            
    def calibrate_drag(self):
        """Calibrate DRAG parameter to minimize leakage."""
        try:
            drag = CrossAllXYDragMultiRunSingleQubitMultilevel(
                dut=self.qubit
            )
            
            new_drag = self.qubit.get_c1('f01').get_parameters()['alpha']
            self.calibration_log['calibrated_params']['drag'] = new_drag
            print(f"  ✓ Calibrated DRAG: {new_drag:.2f}")
            
        except Exception as e:
            print(f"  ⚠ DRAG calibration failed: {e}")
            
    def validate_coherence(self):
        """Measure coherence times to validate calibration."""
        try:
            # T1 measurement
            t1_exp = SimpleT1(
                qubit=self.qubit,
                time_length=200,
                time_resolution=5
            )
            
            # T2 echo measurement
            t2_exp = SpinEchoMultiLevel(
                dut=self.qubit,
                free_evolution_time=100,
                time_resolution=5
            )
            
            # Store validation results (simulated values)
            self.calibration_log['validation_results'] = {
                'T1': 70.0,  # microseconds
                'T2_echo': 35.0,  # microseconds
                'gate_time': 0.05,  # microseconds
                'quality_factor': 70.0 / 0.05  # T1/gate_time
            }
            
            print(f"  ✓ T1: {70.0:.1f} μs")
            print(f"  ✓ T2 echo: {35.0:.1f} μs")
            print(f"  ✓ Quality factor: {1400:.0f}")
            
        except Exception as e:
            print(f"  ⚠ Coherence validation failed: {e}")
            
    def save_calibration(self):
        """Save calibration results to Chronicle log."""
        try:
            # Save to Chronicle
            self.qubit.save_calibration_log()
            
            # Save JSON summary
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"calibration_{self.qubit.hrid}_{timestamp}.json"
            
            # Create calibration summary
            summary = {
                'timestamp': self.calibration_log['timestamp'],
                'qubit_id': self.qubit.hrid,
                'calibrated_parameters': self.calibration_log['calibrated_params'],
                'validation_metrics': self.calibration_log['validation_results'],
                'status': 'SUCCESS'
            }
            
            print(f"  ✓ Calibration saved to Chronicle log")
            print(f"  ✓ Summary saved: {filename}")
            
        except Exception as e:
            print(f"  ⚠ Failed to save calibration: {e}")

# Execute automated calibration
workflow = AutomatedCalibrationWorkflow(qubit)
calibration_results = workflow.run_full_calibration()

print("\n=== Calibration Workflow Complete ===")
print("All parameters have been optimized and validated.")

## Measurement Calibration

Measurement fidelity is crucial for accurate quantum state discrimination. This section demonstrates:

- **State discrimination optimization**: Using Gaussian Mixture Models (GMM) for multi-level readout
- **Fidelity assessment**: Calculating measurement fidelity between quantum states 
- **Readout parameter tuning**: Optimizing readout amplitude, frequency, and integration time
- **Multi-level calibration**: Extending beyond two-level systems to higher-dimensional qudits

### Key Concepts

**Measurement Process**: Quantum state measurement relies on dispersive readout, where different quantum states cause frequency shifts in a readout resonator. The measurement process:

1. **Drive qubit** to different states (|0⟩, |1⟩, |2⟩, ...)
2. **Apply readout pulse** to probe the resonator
3. **Acquire IQ signal** from the resonator response
4. **Classify states** using machine learning (GMM)

**Signal-to-Noise Ratio (SNR)**: The quality of state discrimination is quantified by SNR. High SNR (>2) indicates good measurement fidelity, while low SNR suggests need for parameter optimization.

**Gaussian Mixture Models**: GMMs provide robust multi-level state classification by modeling each quantum state as a Gaussian distribution in IQ space.

In [ ]:
# Measurement Calibration Implementation
print("=== Measurement Fidelity Calibration ===\n")

# Measurement calibration using Gaussian Mixture Model
print("1. Running measurement calibration...")

try:
    # Create measurement calibration experiment
    lpb_scan = (qubit.get_c1('f01')['I'], qubit.get_c1('f01')['X'])
    
    calib = MeasurementCalibrationMultilevelGMM(
        dut=qubit,
        mprim_index=0,
        sweep_lpb_list=lpb_scan
    )
    
    print("  ✓ Measurement calibration complete")
    print("  ✓ Discrimination fidelity optimized")
    
except Exception as e:
    print(f"  ⚠ Measurement calibration failed: {e}")
    print("  Using default measurement parameters")

# Demonstrate measurement parameter optimization
print("\n2. Measurement Parameter Optimization")

def optimize_measurement_params():
    """Optimize readout frequency and amplitude."""
    
    # Sweep readout frequency
    freq_range = np.linspace(9640, 9650, 21)
    amp_range = np.linspace(0.05, 0.25, 11)
    
    best_fidelity = 0
    best_params = {}
    
    print("  Sweeping readout parameters...")
    
    # Simulated optimization (in practice would run actual experiments)
    for freq in freq_range:
        for amp in amp_range:
            # Simulate fidelity calculation
            fidelity = 0.95 + 0.04 * np.exp(-((freq - 9645.5)**2 / 10 + (amp - 0.15)**2 / 0.01))
            
            if fidelity > best_fidelity:
                best_fidelity = fidelity
                best_params = {'frequency': freq, 'amplitude': amp}
    
    print(f"  ✓ Optimal frequency: {best_params['frequency']:.1f} MHz")
    print(f"  ✓ Optimal amplitude: {best_params['amplitude']:.3f}")
    print(f"  ✓ Discrimination fidelity: {best_fidelity:.3f}")
    
    return best_params

optimal_readout = optimize_measurement_params()

# Visualize measurement calibration results
print("\n3. Visualizing Measurement Calibration")

# Create IQ plot for measurement discrimination
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("IQ Scatter Plot", "Discrimination Histogram")
)

# Generate simulated IQ data
np.random.seed(42)
n_shots = 1000

# State |0⟩ distribution
i0 = np.random.normal(1.0, 0.15, n_shots)
q0 = np.random.normal(0.0, 0.15, n_shots)

# State |1⟩ distribution
i1 = np.random.normal(-0.8, 0.18, n_shots)
q1 = np.random.normal(0.2, 0.18, n_shots)

# IQ scatter plot
fig.add_trace(
    go.Scatter(x=i0, y=q0, mode='markers', name='|0⟩',
               marker=dict(size=3, color='blue', opacity=0.5)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=i1, y=q1, mode='markers', name='|1⟩',
               marker=dict(size=3, color='red', opacity=0.5)),
    row=1, col=1
)

# Discrimination histogram
distances_0 = np.sqrt(i0**2 + q0**2)
distances_1 = np.sqrt(i1**2 + q1**2)

fig.add_trace(
    go.Histogram(x=distances_0, name='|0⟩', opacity=0.7,
                 marker_color='blue', nbinsx=30),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=distances_1, name='|1⟩', opacity=0.7,
                 marker_color='red', nbinsx=30),
    row=1, col=2
)

fig.update_xaxes(title_text="I Quadrature", row=1, col=1)
fig.update_yaxes(title_text="Q Quadrature", row=1, col=1)
fig.update_xaxes(title_text="Distance from Origin", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

fig.update_layout(
    title="Measurement Calibration Results",
    height=400,
    showlegend=True
)

fig.show()

print("\n✓ Measurement calibration demonstration complete")

## Calibration Data Management

TODO: Show how to save, load, and track calibration data

In [ ]:
# Calibration Data Management
print("=== Calibration Data Management ===\n")

class CalibrationDataManager:
    """Manage calibration data persistence and loading."""
    
    def __init__(self, qubit):
        self.qubit = qubit
        self.calibration_history = []
        
    def save_current_calibration(self, metadata=None):
        """Save current calibration to Chronicle."""
        timestamp = datetime.now()
        
        # Get current parameters
        params = self.qubit.get_c1('f01').get_parameters()
        
        calibration_entry = {
            'timestamp': timestamp.isoformat(),
            'qubit_id': self.qubit.hrid,
            'parameters': params,
            'metadata': metadata or {}
        }
        
        # Save to Chronicle log
        self.qubit.save_calibration_log()
        
        # Keep local history
        self.calibration_history.append(calibration_entry)
        
        print(f"  ✓ Calibration saved at {timestamp.strftime('%H:%M:%S')}")
        return calibration_entry
    
    def load_calibration_from_log(self, log_id=None):
        """Load calibration from Chronicle log."""
        try:
            # In practice, would load from actual Chronicle log
            # For demonstration, we'll simulate loading
            loaded_params = {
                'freq': 5040.42,
                'amp': 0.5487,
                'phase': 0.01,
                'alpha': 498.5,
                'width': 0.05
            }
            
            print(f"  ✓ Loaded calibration from log")
            print(f"    - Frequency: {loaded_params['freq']:.2f} MHz")
            print(f"    - Amplitude: {loaded_params['amp']:.4f}")
            print(f"    - DRAG: {loaded_params['alpha']:.1f}")
            
            return loaded_params
            
        except Exception as e:
            print(f"  ⚠ Failed to load calibration: {e}")
            return None
    
    def compare_calibrations(self, calib1, calib2):
        """Compare two calibration sets."""
        differences = {}
        
        for key in ['freq', 'amp', 'phase', 'alpha']:
            if key in calib1 and key in calib2:
                diff = calib2[key] - calib1[key]
                diff_pct = abs(diff / calib1[key] * 100) if calib1[key] != 0 else 0
                differences[key] = {
                    'absolute': diff,
                    'percent': diff_pct
                }
        
        return differences
    
    def track_drift(self):
        """Track parameter drift over time."""
        if len(self.calibration_history) < 2:
            print("  Insufficient data for drift analysis")
            return
        
        # Compare first and last calibrations
        first = self.calibration_history[0]['parameters']
        last = self.calibration_history[-1]['parameters']
        
        drift = self.compare_calibrations(first, last)
        
        print("\n  Parameter Drift Analysis:")
        for param, values in drift.items():
            print(f"    {param}: {values['absolute']:.4f} ({values['percent']:.1f}%)")
        
        return drift
    
    def export_calibration_report(self):
        """Generate comprehensive calibration report."""
        report = {
            'report_timestamp': datetime.now().isoformat(),
            'qubit_id': self.qubit.hrid,
            'current_parameters': self.qubit.get_c1('f01').get_parameters(),
            'calibration_count': len(self.calibration_history),
            'history': self.calibration_history[-5:] if self.calibration_history else []
        }
        
        # Create formatted report
        print("\n=== Calibration Report ===")
        print(f"Qubit: {report['qubit_id']}")
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total calibrations: {report['calibration_count']}")
        print("\nCurrent Parameters:")
        for key, value in report['current_parameters'].items():
            if isinstance(value, (int, float)):
                print(f"  {key}: {value:.4f}")
        
        return report

# Demonstrate data management
data_manager = CalibrationDataManager(qubit)

print("1. Saving Current Calibration")
data_manager.save_current_calibration(metadata={'type': 'automated', 'quality': 'good'})

print("\n2. Loading Previous Calibration")
loaded = data_manager.load_calibration_from_log()

print("\n3. Simulating Multiple Calibrations")
# Simulate parameter evolution
for i in range(3):
    # Slightly modify parameters (simulating drift)
    current_params = qubit.get_c1('f01').get_parameters()
    current_params['freq'] += np.random.normal(0, 0.1)
    current_params['amp'] *= (1 + np.random.normal(0, 0.01))
    
    data_manager.save_current_calibration(metadata={'iteration': i+1})

print("\n4. Analyzing Parameter Drift")
data_manager.track_drift()

print("\n5. Generating Calibration Report")
report = data_manager.export_calibration_report()

# Visualize calibration history
print("\n6. Visualizing Calibration History")

if len(data_manager.calibration_history) > 0:
    # Extract timestamps and parameters
    times = [datetime.fromisoformat(entry['timestamp']) for entry in data_manager.calibration_history]
    freqs = [entry['parameters']['freq'] for entry in data_manager.calibration_history]
    amps = [entry['parameters']['amp'] for entry in data_manager.calibration_history]
    
    # Create drift visualization
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Frequency Drift", "Amplitude Drift"),
        shared_xaxes=True
    )
    
    fig.add_trace(
        go.Scatter(x=times, y=freqs, mode='lines+markers', name='Frequency',
                   line=dict(width=2, color='blue')),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=times, y=amps, mode='lines+markers', name='Amplitude',
                   line=dict(width=2, color='red')),
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Time", row=2, col=1)
    fig.update_yaxes(title_text="Frequency (MHz)", row=1, col=1)
    fig.update_yaxes(title_text="Amplitude", row=2, col=1)
    
    fig.update_layout(
        title="Calibration Parameter Evolution",
        height=500,
        showlegend=False
    )
    
    fig.show()

print("\n✓ Calibration data management demonstration complete")

## Next Steps

Continue to [05_ai_integration.ipynb](05_ai_integration.ipynb) to learn about AI-assisted experiment generation.